# Linear Regression in Pandas

Testing basic linear and logistic regression with OHE of some variables from the dataset.

### Setup

In [1]:
# Extension reloader to import a function again when re-running cell 
%load_ext autoreload
%autoreload 2

### Load Configuration

In [2]:
"""
Loads common configuration parameters
"""
from src.utils.configuration_manager import Config
from pathlib import Path, PurePath
from os import getcwd

config_path = PurePath(getcwd(),'config.ini')
config = Config(config_path)

Loading configuration from: /home/justin/Code/interpretability_experiment/config.ini


### Start local Dask Client

In [3]:
from dask.distributed import Client, LocalCluster
try:
    if client:
        print('Restarting client')
        client.restart()
except:
    cluster = LocalCluster(dashboard_address=':20100', memory_limit='4G')
#     cluster = LocalCluster(dashboard_address=':20100')
    print('Setting new client')
    client = Client(cluster)
    print(client)
client

Setting new client
<Client: 'tcp://127.0.0.1:39275' processes=5 threads=10, memory=20.00 GB>


Client Scheduler: tcp://127.0.0.1:39275 Dashboard: http://127.0.0.1:20100/status,Cluster Workers: 5 Cores: 10 Memory: 20.00 GB


## Prepare / Load dataset

In [4]:
from src.preprocessing.dataset_manager import DatasetManager
dataset_manager = DatasetManager(config)

In [5]:
%time dataset_manager.write_dataset(overwrite=False)

Not overwriting existing training and test sets
CPU times: user 363 µs, sys: 68 µs, total: 431 µs
Wall time: 249 µs


In [6]:
X_train, y_train = dataset_manager.get_training_set()
X_test, y_test = dataset_manager.get_test_set()

### Additional Preprocessing Step

In [7]:
categorical_columns_to_transform = ['payment_type']
# datetime_columns_to_transform = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
datetime_columns_to_transform = []

In [8]:
from dask_ml.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)

In [9]:
import src.preprocessing.preprocessing_pipelines as preprocessing

def preprocess(ddf):
    return preprocessing.ohe_preprocessing_pipeline(encoder, 
                                     ddf, 
                                     categorical_columns_to_transform = categorical_columns_to_transform, 
                                     datetime_columns_to_transform = datetime_columns_to_transform)

X_train = preprocess(X_train)
X_test = preprocess(X_test)

In [10]:
input_columns = X_train.columns.values

In [11]:
X_train.head()

,passenger_count,trip_distance,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount_wo_tip,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5
index,,,,,,,,,,,,,
0,1.0,1.20,11.0,0.0,0.5,0.00,0.3,11.80,1.0,0.0,0.0,0.0,0.0
1,1.0,12.03,39.0,0.0,0.5,5.76,0.3,45.56,1.0,0.0,0.0,0.0,0.0
2,2.0,0.86,5.5,0.0,0.5,0.00,0.3,6.30,1.0,0.0,0.0,0.0,0.0
3,2.0,1.09,6.0,0.0,0.5,0.00,0.3,6.80,1.0,0.0,0.0,0.0,0.0
5,2.0,1.32,10.0,0.0,0.5,0.00,0.3,10.80,1.0,0.0,0.0,0.0,0.0


### Training Linear Regression with Pandas

In [12]:
target = 'tip_fraction'
partitions_to_concat = 20 
import src.preprocessing.dask_to_pandas as dtp

In [13]:
from sklearn.linear_model import LinearRegression

import joblib

def train_linear_regression(X_ddf, y_ddf, target, partitions_to_concat, save_to):
    """
    In memory training of linear regression
    """
    X, y = dtp.dask_Xy_to_df(X_ddf, y_ddf, target, partitions_to_concat)
    
    estimator = LinearRegression()
#     estimator = Lasso()
    estimator.fit(X, y=y)
    
    joblib.dump(estimator, save_to)
    return estimator

In [14]:
linear_reg_save = 'trained_models/linr_estimator_w_cat.sav'
lr_estimator = train_linear_regression(X_train, y_train, target, partitions_to_concat, linear_reg_save)

### Test Linear Regression

In [15]:
import numpy as np
import eli5

/home/justin/.conda/envs/ml_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/justin/.conda/envs/ml_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [16]:
def test_regression(X_ddf, y_ddf, target, estimator):
    """
    In memory testing of regression model
    """
    X, y = dtp.dask_Xy_to_df(X_ddf, y_ddf, target, partitions_to_concat)
    
    predictions = estimator.predict(X)
    print('MAE: ' + str(abs(predictions-y).mean(axis=0)))
    print('RMSE: ' + str(np.sqrt(((predictions-y)**2).mean(axis=0))))
    

In [17]:
# Normal linear regression
test_regression(X_test, y_test, target, lr_estimator)

MAE: 0.0359856289841919
RMSE: 0.5305681445646424


In [18]:
eli5.explain_weights(lr_estimator, feature_names=input_columns, top=20)

Weight?,Feature
+123980290.696,payment_type_1
+123980290.505,payment_type_2
+123980290.472,payment_type_3
+123980290.448,payment_type_4
+0.016,improvement_surcharge
+0.016,fare_amount
+0.015,extra
+0.014,tolls_amount
+0.000,passenger_count
-0.001,trip_distance


In [19]:
# TODO resolve bug that has previous targets values bleed into training set
# print(X_train.columns.values)
# Patch the underlying bug
X_train = X_train.drop(columns = target, axis=1)

### Training Logistic Regression with Pandas

In [20]:
target = 'tipped'

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
def train_logistic_regression(X_ddf, y_ddf, target, partitions_to_concat, save_to):
    """
    In memory training of linear regression
    """
    X, y = dtp.dask_Xy_to_df(X_ddf, y_ddf, target, partitions_to_concat)
        
    estimator = LogisticRegression()
    estimator.fit(X, y=y)
    
    joblib.dump(estimator, save_to)
    return estimator

In [23]:
logistic_reg_save = 'trained_models/logr_estimator_w_cat.sav'
logr_estimator = train_logistic_regression(X_train, y_train, target, partitions_to_concat, logistic_reg_save)

/home/justin/.conda/envs/ml_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Test Logistic Regression

In [24]:
from sklearn.metrics import accuracy_score
def test_classification(X_ddf, y_ddf, target, estimator):
    """
    In memory testing of linear regression 
    """
    X, y = dtp.dask_Xy_to_df(X_ddf, y_ddf, target, partitions_to_concat)
    
    predictions = estimator.predict(X)
    print(accuracy_score(predictions, y))

# TODO Find bug that is adding the target to X_train without removing it after

In [25]:
test_classification(X_train, y_train, target, logr_estimator)

0.9671217596318146


In [26]:
eli5.explain_weights(logr_estimator, feature_names=input_columns, top=20)

Weight?,Feature
+6.778,payment_type_1
+0.870,mta_tax
+0.624,total_amount_wo_tip
+0.008,passenger_count
-0.062,trip_distance
-0.403,extra
-0.557,tolls_amount
-0.620,improvement_surcharge
-0.639,fare_amount
-0.915,payment_type_4
